<h1>Part 2: Data Analysis of Rent Price Data</h1>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()

In [ ]:
df = pd.read_csv('data/renthub_data.csv')

In [ ]:
df

In [ ]:
# Delete unwanted colomn to save on processing time
del df['marketing_desc']

In [ ]:
df.describe()

## Dealing with duplicate values
For duplicate values, we need to delete only the ones that have same posted dates, since a unit can be listed multiple times over years. 

In [ ]:
df = df.drop_duplicates(
    subset = ['address', 'sqft', 'lat', 'long', 'posted_at'],
    keep = 'first').reset_index(drop = True)

In [ ]:
df

Our new dataset contains much lesser number of rows. It is around 1/10th of our original data. 

## Exploratory analysis
Lets see the stats for the difference between the date posted and the 

In [ ]:
df['posted_at'] = pd.to_datetime(df['posted_at'])
df['scraped_at'] = pd.to_datetime(df['scraped_at'])
df['posted_difference'] = df['scraped_at'] - df['posted_at']
print('99th percentile: ', df['posted_difference'].quantile(0.99))
df['posted_difference'].describe()

With a standard deviation of around 15 days, units with posted difference of greater than 90 days can be considered as outliers and be removed

In [ ]:
df = df[df['posted_difference'].astype('timedelta64[D]') <= 90]

In [ ]:
df

<br>Not all zip codes contain the same number of houses, so we need to remove the ones that contain number of houses less than a certain threshold. To get this threshold, we will need to visualize this dataset.

In [ ]:
counts = df.groupby(by="zip").id.count()
print(len(counts))
counts = counts[counts < 10]
_ = plt.hist(counts)

Out of around 60,000 zipcodes in our dataset, around 40,000 have 2 or lesser number of entries. We will be considering only the ones that have 3 or more entires since the graphs that we are targeting need to have multiple entries. Also for aggregation metrics, all types of metrics can be dominated by outliers for even 2 entires and so we need at least 3. This may look like dropping a lot of rows, but in reality they make up a really small part of our dataset. 

In [ ]:
df = df[~df['zip'].isin(counts[counts < 3].index)]

In [ ]:
df

This is not enough. We also need to remove the zip codes that have just one distinct month in them, since the intended visualization in that case will be just be a single point. 

In [ ]:
df['posted_month'] = df['posted_at'].dt.date + pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin(1)
df_zip_month = df[['id', 'zip', 'posted_month']].drop_duplicates()
zip_month_counts = df_zip_month.groupby('zip')['id'].count()
zip_month_counts.head()

In [ ]:
zip_month_counts = zip_month_counts[zip_month_counts >= 2]
df = df[~df['zip'].isin(zip_month_counts.index)]
df.head()

## Cleaning
Any row will nan in the zip column or posted at column needs to be removed. <br> We have few rows that have non alphanumeric characters in zip code column, so we need to remove those rows.

In [ ]:
df = df[df.zip.notna()]
df = df[df.posted_at.notna()]
df = df[~df.zip.str.contains(r'[^\w\s]', na=False)]

## Create featues for appropirate graphs
Below given function transforms the data in the best way posible to be visualized in the format provided in the example. It takes the zip code we want to visialize for as an argument. We can make multiple calls in case of multiple zip codes. 

In [ ]:
def aggregate_on_attribute(df, zip_code, col, agg='mean'):
    df = df[df['zip'] == zip_code]
    df_1 = df[['id', col, 'posted_at', 'zip']]
    df_1['month'] = (df_1['posted_at'] + pd.offsets.MonthBegin(1)).dt.date
    df_2 = df[['id', col, 'posted_at', 'zip']]
    df_2['month'] = (df_2['posted_at'] + pd.offsets.MonthBegin(2)).dt.date
    df_3 = df[['id', col, 'posted_at', 'zip']]
    df_3['month'] = (df_3['posted_at'] + pd.offsets.MonthBegin(3)).dt.date
    df_features = pd.concat([df_1, df_2, df_3])
    if agg == 'mean':
        grouped_df = df_features.groupby(by='month')[col].mean()
    elif agg == 'max':
        grouped_df = df_features.groupby(by='month')[col].max()
    elif agg == 'min':
        grouped_df = df_features.groupby(by='month')[col].min()
    else:
        print('agg can be one of mean, min, max')
        return
    return grouped_df
df_features = aggregate_on_attribute(df, 10001, 'price', 'mean')
df_features

In [ ]:
plt.plot(df_features)

In [ ]:
# for multiple zip codes

for zip_code in [85741, 85716, 85718, 85711, 85712]:
    plt.plot(aggregate_on_attribute(df, zip_code, 'price', 'mean'))

Summary:
- The ETL pipeline given above reads data, removes all unwanted rows, features and duplicates, and finally puts in such a format that visualization is easy. 